In [21]:
import geopandas as gpd
import numpy as np

In [22]:
driving = gpd.read_file('../data/googleQueryDriving/googleQueryDriving.shp')
transit = gpd.read_file('../data/googleQueryTransit/googleQuery.shp')
data = gpd.read_file('../data/radiosQuearyCABA/radiosQuearyCABA.shp')
data.drop(['mejorMod','x','y'],axis=1,inplace=True)

In [23]:
data['transitGoog'] = transit.tiempos*1./60 
data['geometry'] = transit.geometry
data['transitDineroGoog'] = transit.dinero 

data['driveGoog'] = driving.tiempos*1./60 
data['traficGoog'] = driving.trafico*1./60 
data['driveTot'] = data.traficGoog + data.driveGoog


In [24]:
data['traficPerc'] = data.traficGoog/data.driveTot
data['bikePerc'] = data.tiemposBic / data.driveTot
data['bikeVtrafic'] = data.tiemposBic / data.transitGoog.replace(to_replace=0, value=1)



In [25]:
data.head()

,CO_FRAC_RA,geometry,tiemposAut,tiemposBic,tiemposTra,transitGoog,transitDineroGoog,driveGoog,traficGoog,driveTot,traficPerc,bikePerc,bikeVtrafic
0,1_1_1,POLYGON ((-58.37189352634645 -34.5919935640870...,10.0,15.0,14.0,26.683333,6.00,10.683333,16.150000,26.833333,0.601863,0.559006,0.562149
1,1_12_1,POLYGON ((-58.38592548750878 -34.6044398706025...,10.0,12.0,8.0,12.516667,6.25,10.516667,12.233333,22.750000,0.537729,0.527473,0.958722
2,1_12_10,POLYGON ((-58.37878669239328 -34.6052849003551...,6.0,4.0,8.0,4.566667,0.00,8.350000,9.700000,18.050000,0.537396,0.221607,0.875912
3,1_12_11,POLYGON ((-58.36732497447693 -34.6056209207810...,2.0,1.0,2.0,1.616667,0.00,0.616667,0.600000,1.216667,0.493151,0.821918,0.618557
4,1_12_2,POLYGON ((-58.38453585809192 -34.6043583533245...,7.0,9.0,8.0,9.850000,7.50,7.483333,9.300000,16.783333,0.554121,0.536246,0.913706


In [26]:
data.tiemposBic.replace(to_replace=0, value=np.nan, inplace=True)

In [27]:
#better mode
valores = data.loc[:,['tiemposBic','driveTot','transitGoog']]
minvalues = valores.values.min(axis=1)
better=[ ]

for i in range(data.shape[0]):
    if np.isnan(minvalues[i]):
        mejor = np.nan
    else:
        mejor = valores.columns[valores.loc[i] == minvalues[i]].values[0]
    better.append(mejor)

In [28]:
data['mejorMod'] = better

In [29]:
data.mejorMod.value_counts()

transitGoog    2420
tiemposBic      692
driveTot        361
Name: mejorMod, dtype: int64

In [30]:
data.crs

{'init': u'epsg:4326'}

In [33]:
data = data.to_crs(epsg=22185)

In [34]:
data.to_file('../data/basefinal/')

In [20]:
data.bikeVtrafic.describe()

count    3473.000000
mean        1.290987
std         0.344073
min         0.304878
25%         1.058540
50%         1.299639
75%         1.527531
max         2.520730
Name: bikeVtrafic, dtype: float64